In [23]:
import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM, BartForConditionalGeneration,BartTokenizer,AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import shap
import scipy as sp
import nlp
import torch
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)
from shap.utils import assert_import, record_import_error, safe_isinstance, make_masks, OpChain, MaskedModel
from shap import maskers, links
from transformers.file_utils import ModelOutput
#from shap.utils import cal_conditional_logits
import sys
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})

In [57]:
tokenizer = AutoTokenizer.from_pretrained("ishan/distilbert-base-uncased-mnli")

model = AutoModelForSequenceClassification.from_pretrained("ishan/distilbert-base-uncased-mnli")

In [123]:
sent1="A study of Brazil nuts in the Amazon has suggested that intensive harvesting could adversely affect the natural regeneration of the trees."
sent2="A study of Brazil nuts in the Amazon has revealed that seasonal food availability affects the dispersal of the seeds."


In [124]:
encoding = tokenizer(sent1, sent2, return_tensors='pt')

In [125]:
with torch.no_grad():
    outputs = model(**encoding)[0]

In [126]:
m=torch.nn.Softmax(dim=1)
probs = m(outputs)
probs

tensor([[0.6270, 0.0155, 0.3575]])

In [96]:
scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T

In [97]:
scores

tensor([[0.0134, 0.8547, 0.1318]])

In [118]:
def f(x,**kwargs):
    output_batch=[]
    for sent1 in x:
        encoding = tokenizer(sent1, sent2, return_tensors='pt')
        #attention_mask = (tv!=0).type(torch.int64).cuda()
        with torch.no_grad():
            outputs = model(**encoding)[0][0].numpy()
        scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        val = sp.special.logit(scores) 
        output_batch.append(val)
    #print(output_batch)
    return output_batch

In [119]:
f(['A study of Brazil nuts in the Amazon has suggested that intensive harvesting could adversely affect the natural regeneration of the trees.'])

[array([0.51924, -4.15214, -0.58604], dtype=float32)]

In [120]:
explainer = shap.Explainer(f,tokenizer)

explainers.Partition is still in an alpha state, so use with caution...


In [121]:
shap_values_multiclass = explainer([sent1])

Index in batch: 50
Interaction effect for index 50: [0.00000 0.00000 0.00000]
Index in batch: 50
Interaction effect for index 49: [0.00000 0.00000 0.00000]
Index in batch: 49
Interaction effect for index 48: [-2.07671 2.00387 1.43674]
Index in batch: 48
Interaction effect for index 47: [-0.20546 1.37945 0.12339]
Interaction effect for index 46: [-0.37273 1.15028 0.57017]
Index in batch: 46
Interaction effect for index 45: [-0.01321 1.93469 -0.32100]
Interaction effect for index 42: [0.39056 0.56434 -0.31308]
Interaction effect for index 43: [0.11068 -0.04910 -0.04009]
Index in batch: 43
Interaction effect for index 40: [0.28048 0.24383 -0.13661]
Interaction effect for index 35: [0.17239 -0.24479 -0.07940]
Interaction effect for index 44: [-0.34799 4.16241 -0.45979]
Index in batch: 44
Interaction effect for index 39: [0.54491 0.04454 -0.34866]
Interaction effect for index 38: [0.23962 -0.34595 -0.04529]
Interaction effect for index 32: [-0.11718 -0.20090 0.25781]
Index in batch: 32
Inte

In [128]:
shap.plots.text(shap_values_multiclass[0][:,1],group_threshold=1)

In [ ]:
"A study of Brazil nuts in the Amazon has revealed that seasonal food availability affects the dispersal of the seeds."